In [3]:
library(lmerTest)
library(MuMIn)
library(VIF)
library(sqldf)

filename <- 'generated/metrics_for_time_series.csv'

TimeSeriesData <- read.csv(file=filename, header=TRUE, sep=",")

summary(TimeSeriesData)

TimeSeriesData <- sqldf("select *
                      from 'TimeSeriesData' 
                      where Period != 13")


TimeSeriesData$Intervention = TimeSeriesData$Period > 12
TimeSeriesData$TimeAfterIntervention = ifelse(TimeSeriesData$Period>12, TimeSeriesData$Period-12, 0)

Loading required package: Matrix
Loading required package: lme4

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

Loading required package: gsubfn
Loading required package: proto
Warning message:
“no DISPLAY variable so Tk is not available”Loading required package: RSQLite


                       ProjectName         Language   ShareReviewComments
 AFNetworking/AFNetworking   : 25   C++        :200   Min.   :0.0000     
 AnalyticalGraphicsInc/cesium: 25   Java       :100   1st Qu.:0.0000     
 antlr/antlr4                : 25   JavaScript :100   Median :0.1621     
 apache/storm                : 25   Python     : 75   Mean   :0.2292     
 appcelerator/alloy          : 25   Objective-C: 50   3rd Qu.:0.3632     
 Automattic/_s               : 25   C#         : 25   Max.   :0.9634     
 (Other)                     :475   (Other)    : 75                      
   Additions         Deletions        ChangedFiles       Assignees     
 Min.   :      0   Min.   :      0   Min.   :    0.0   Min.   : 0.000  
 1st Qu.:    713   1st Qu.:    225   1st Qu.:   40.0   1st Qu.: 0.000  
 Median :   2864   Median :   1146   Median :  109.0   Median : 0.000  
 Mean   :  41546   Mean   :  16043   Mean   :  387.1   Mean   : 1.314  
 3rd Qu.:  11377   3rd Qu.:   4621   3rd Qu.:  2

In [4]:
vif.mer <- function (fit) {
    ## adapted from rms::vif
    
    v <- vcov(fit)
    nam <- names(fixef(fit))

    ## exclude intercepts
    ns <- sum(1 * (nam == "Intercept" | nam == "(Intercept)"))
    if (ns > 0) {
        v <- v[-(1:ns), -(1:ns), drop = FALSE]
        nam <- nam[-(1:ns)]
    }
    
    d <- diag(v)^0.5
    v <- diag(solve(v/(d %o% d)))
    names(v) <- nam
    v
}

In [5]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesData")

print(projects)

  count(distinct(ProjectName))
1                           25


In [6]:
modelNumberReviewComments = lmer(log(ReviewComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(TotalPrs) +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelNumberReviewComments)
r.squaredGLMM(modelNumberReviewComments)
vif.mer(modelNumberReviewComments)
anova(modelNumberReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ReviewComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(TotalPrs) + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
  1665.4   1731.3   -817.7   1635.4      585 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9820 -0.5585 -0.0258  0.6072  3.3021 

Random effects:
 Groups      Name             Variance Std.Dev. Corr
 ProjectName (Intercept)      1.2093   1.0997       
             InterventionTRUE 0.3125   0.5590   0.03
 Language    (Intercept)      0.0000   0.0000       
 Residual                     0.7254   0.8517       
Number of obs: 600, groups:  ProjectName, 25; Language, 9

Fixed effects:
                        Estimate

R2m       R2c 
0.2513170 0.7422856

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.916048              3.774991              2.727318 
   log(Assignees + 1) log(ChangedFiles + 1)         log(TotalPrs) 
             1.108220              5.978997              2.093465 
     InterventionTRUE                Period TimeAfterIntervention 
             2.049259              4.097408              3.410962

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),2.415377705,2.415377705,1,579.91598,3.329874263,6.854639e-02
log(Deletions + 1),2.948923871,2.948923871,1,568.05700,4.065428641,4.423974e-02
log(Commits + 1),11.938390964,11.938390964,1,585.17688,16.458436593,5.647289e-05
log(Assignees + 1),5.648993032,5.648993032,1,587.40847,7.787782617,5.430972e-03
log(ChangedFiles + 1),0.006030321,0.006030321,1,575.32419,0.008313487,9.273826e-01
log(TotalPrs),21.151408839,21.151408839,1,577.11906,29.159634850,9.753289e-08
Intervention,0.561439002,0.561439002,1,98.93742,0.774007841,3.811120e-01
Period,7.676133730,7.676133730,1,550.95645,10.582427787,1.211589e-03
TimeAfterIntervention,2.000292510,2.000292510,1,551.37491,2.757631874,9.735993e-02


In [7]:
modelShareReviewComments = lmer(log(ShareReviewComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(TotalPrs) +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(ShareReviewComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(TotalPrs) + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
  -846.0   -780.1    438.0   -876.0      585 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9497 -0.5225 -0.0739  0.4555  4.0681 

Random effects:
 Groups      Name             Variance Std.Dev. Corr
 ProjectName (Intercept)      0.011712 0.10822      
             InterventionTRUE 0.005233 0.07234  0.28
 Language    (Intercept)      0.000000 0.00000      
 Residual                     0.011218 0.10592      
Number of obs: 600, groups:  ProjectName, 25; Language, 9

Fixed effects:
                        Est

R2m        R2c 
0.08139258 0.62821790

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.924360              3.798359              2.704164 
   log(Assignees + 1) log(ChangedFiles + 1)         log(TotalPrs) 
             1.107455              6.006385              2.054110 
     InterventionTRUE                Period TimeAfterIntervention 
             1.989320              3.981977              3.333917

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),0.037679608,0.037679608,1,583.55777,3.3588465,6.735421e-02
log(Deletions + 1),0.031149973,0.031149973,1,571.50736,2.7767798,9.618822e-02
log(Commits + 1),0.201637402,0.201637402,1,571.53740,17.9744193,2.610674e-05
log(Assignees + 1),0.025800071,0.025800071,1,595.62380,2.2998774,1.299147e-01
log(ChangedFiles + 1),0.009304004,0.009304004,1,579.17940,0.8293802,3.628307e-01
log(TotalPrs),0.001972951,0.001972951,1,531.08099,0.1758734,6.751134e-01
Intervention,0.008207643,0.008207643,1,93.78221,0.7316480,3.945295e-01
Period,0.091569673,0.091569673,1,551.40581,8.1627301,4.437367e-03
TimeAfterIntervention,0.049035608,0.049035608,1,551.86980,4.3711463,3.700925e-02


In [8]:
modelShareReviewComments = lmer(log(GeneralComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            log(TotalPrs) +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(GeneralComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + log(TotalPrs) + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
  1067.1   1133.0   -518.5   1037.1      585 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9791 -0.4745  0.0650  0.6123  3.2999 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.4641   0.6812        
             InterventionTRUE 0.1612   0.4015   -0.58
 Language    (Intercept)      0.0000   0.0000        
 Residual                     0.2689   0.5186        
Number of obs: 600, groups:  ProjectName, 25; Language, 9

Fixed effects:
                        Es

R2m       R2c 
0.5490647 0.8144814

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.945606              3.781725              2.795218 
   log(Assignees + 1) log(ChangedFiles + 1)         log(TotalPrs) 
             1.100507              6.010120              2.154788 
     InterventionTRUE                Period TimeAfterIntervention 
             1.818573              3.648823              3.110700

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),2.179575e-01,2.179575e-01,1,579.76503,8.104740e-01,0.368353936
log(Deletions + 1),3.856649e-04,3.856649e-04,1,567.48403,1.434093e-03,0.969805108
log(Commits + 1),2.418783e-01,2.418783e-01,1,595.32659,8.994235e-01,0.343321140
log(Assignees + 1),2.136683e+00,2.136683e+00,1,551.48490,7.945248e+00,0.004994532
log(ChangedFiles + 1),8.834162e-02,8.834162e-02,1,576.47976,3.284980e-01,0.566768364
log(TotalPrs),6.504560e+01,6.504560e+01,1,596.89447,2.418718e+02,0.000000000
Intervention,1.666162e-01,1.666162e-01,1,75.31166,6.195617e-01,0.433680331
Period,1.209998e-01,1.209998e-01,1,548.80124,4.499375e-01,0.502646839
TimeAfterIntervention,8.725965e-04,8.725965e-04,1,549.82086,3.244747e-03,0.954595620


# how about if we don't apply any aggregation

In [9]:
filename_no_aggr <- 'generated/metrics_for_time_series_no_aggr.csv'

TimeSeriesDataNoAggr <- read.csv(file=filename_no_aggr, header=TRUE, sep=",")

summary(TimeSeriesDataNoAggr)

TimeSeriesDataNoAggr <- sqldf("select *
                      from 'TimeSeriesDataNoAggr' 
                      where Period != 13")


TimeSeriesDataNoAggr$Intervention = TimeSeriesDataNoAggr$Period > 12
TimeSeriesDataNoAggr$TimeAfterIntervention = ifelse(TimeSeriesDataNoAggr$Period>12, TimeSeriesDataNoAggr$Period-12, 0)

                       ProjectName       Period      EffectiveComments
 caskdata/cdap               : 633   Min.   : 1.00   Min.   :  1.000  
 adobe/brackets              : 542   1st Qu.: 6.00   1st Qu.:  1.000  
 DynamoDS/Dynamo             : 420   Median :12.00   Median :  2.000  
 AnalyticalGraphicsInc/cesium: 192   Mean   :12.66   Mean   :  5.848  
 mne-tools/mne-python        : 190   3rd Qu.:19.00   3rd Qu.:  5.000  
 cakephp/docs                : 183   Max.   :25.00   Max.   :179.000  
 (Other)                     :1200                                    
 ReviewThreads      Additions          Deletions            Commits      
 Min.   :  1.00   Min.   :     0.0   Min.   :     0.00   Min.   :  1.00  
 1st Qu.:  2.00   1st Qu.:    16.0   1st Qu.:     1.00   1st Qu.:  3.00  
 Median :  6.00   Median :   103.0   Median :    13.00   Median :  5.00  
 Mean   : 13.16   Mean   :   839.8   Mean   :   349.88   Mean   : 11.49  
 3rd Qu.: 14.00   3rd Qu.:   390.2   3rd Qu.:    78.25   3rd Q

In [10]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesDataNoAggr")

print(projects)

  count(distinct(ProjectName))
1                           15


In [11]:
modelEffectiveCommentsNoAggr = lmer(log(EffectiveComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            IsMerged + 
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1+Intervention|ProjectName),
          data= TimeSeriesDataNoAggr, 
          REML=FALSE)
summary(modelEffectiveCommentsNoAggr)
r.squaredGLMM(modelEffectiveCommentsNoAggr)
vif.mer(modelEffectiveCommentsNoAggr)
anova(modelEffectiveCommentsNoAggr)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(EffectiveComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + IsMerged + Intervention + Period + TimeAfterIntervention +  
    (1 + Intervention | ProjectName)
   Data: TimeSeriesDataNoAggr

     AIC      BIC   logLik deviance df.resid 
  6836.7   6921.8  -3404.4   6808.7     3194 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3158 -0.6691 -0.1427  0.5350  4.3624 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.02610  0.1615        
             InterventionTRUE 0.02843  0.1686   -0.28
 Residual                     0.48082  0.6934        
Number of obs: 3208, groups:  ProjectName, 15

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            4.510e-

R2m       R2c 
0.2357399 0.2836993

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.325657              2.306295              1.358253 
   log(Assignees + 1) log(ChangedFiles + 1)          IsMergedTrue 
             1.012371              3.974100              1.033055 
     InterventionTRUE                Period TimeAfterIntervention 
             1.884337              3.362066              3.028048

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),5.471543e+01,5.471543e+01,1,3201.15080,113.79492725,0.00000000
log(Deletions + 1),3.805505e+01,3.805505e+01,1,3194.63863,79.14534801,0.00000000
log(Commits + 1),2.284280e+02,2.284280e+02,1,3171.11567,475.07524345,0.00000000
log(Assignees + 1),1.722403e+00,1.722403e+00,1,2577.26345,3.58218481,0.05851425
log(ChangedFiles + 1),1.604281e+00,1.604281e+00,1,3202.45927,3.33651794,0.06785090
IsMerged,3.401663e-01,3.401663e-01,1,3186.22576,0.70746416,0.40035005
Intervention,9.618148e-03,9.618148e-03,1,44.50808,0.02000343,0.88816573
Period,4.175097e-01,4.175097e-01,1,3165.14181,0.86831983,0.35149194
TimeAfterIntervention,2.160129e-01,2.160129e-01,1,3185.75351,0.44925497,0.50273755
